In [ ]:
!pip install shapely
!pip install pyproj

In [ ]:
import pandas as pd
df = pd.read_csv('traffic flow v1.csv', sep=',')

In [ ]:
from shapely import wkt
import pandas as pd
from pyproj import Proj, transform

# 定義一個函數將 WKT 轉換為經緯度
def wkt_to_latlon(wkt_str, source_srs):
    point = wkt.loads(wkt_str)
    # 定義轉換的目標 SRS (這裡假設使用的是 EPSG:4326)
    target_proj = Proj(init='epsg:4326')
    # 進行坐標轉換
    lon, lat = transform(Proj(init=source_srs), target_proj, point.x, point.y)
    return lon, lat

# 定義原始坐標系統的EPSG代碼（這裡假設使用的是EPSG:32633）
source_srs = 'epsg:2263'

# 將 WktGeom 列轉換為經緯度列
df[['Longitude', 'Latitude']] = df.apply(lambda row: pd.Series(wkt_to_latlon(row['WktGeom'], source_srs)), axis=1)
df_filtered = df[['M', 'D', 'HH', 'MM', 'Longitude', 'Latitude', 'Vol']]

# 加固定的年份（2020）欄位
df_filtered['year'] = 2020

# 將 'year'、'M'、'D'、'HH'、'MM' 合併成一個日期欄位 'date'
df_filtered['date'] = pd.to_datetime(df_filtered[['year', 'M', 'D', 'HH', 'MM']].astype(str).agg(' '.join, axis=1), format='%Y %m %d %H %M')

# 刪除原始的 'year'、'M'、'D'、'HH'、'MM' 欄位
df_filtered.drop(['year', 'M', 'D', 'HH', 'MM'], axis=1, inplace=True)

# 根據 'date' 欄位計算星期幾，並新增一欄位 'weekday'
df_filtered['weekday'] = df_filtered['date'].dt.day_name()

In [ ]:
df_filtered.to_csv('traffic flow v2.csv', index=False)

In [ ]:
import geopandas as gpd
from shapely.geometry import Point

grouped_gdf = gpd.GeoDataFrame(grouped_df, geometry=gpd.points_from_xy(grouped_df['Longitude'], grouped_df['Latitude']))

# Perform spatial join
grouped_df_with_neighborhoods = gpd.sjoin(grouped_gdf, neighborhoods, op='within', how='left')

# Check resulting columns
print(grouped_df_with_neighborhoods.columns)

# Drop unnecessary columns
if 'neighborhood' in grouped_df_with_neighborhoods.columns:
    grouped_df_with_neighborhoods = grouped_df_with_neighborhoods[['Longitude', 'Latitude', 'geometry', 'neighborhood']]

grouped_df_with_neighborhoods

In [ ]:
grouped_df_with_neighborhoods.to_csv('traffic flow v3.csv', index=False)